### 마케팅 활동에 대한 분석을 수행하기 위한 2년(2021-2023) 동안 중간 규모 온라인 스토어의 멀티 채널 메시지가 포함된 이커머스 여러채널 다이렉트 메시지 데이터

In [185]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, BulkIndexError

In [171]:
# 데이터 읽기
campaigns_path = r"C:\Users\SEC\Desktop\부트캠프\shopping_i-five\archive\campaigns.csv"
campaigns = pd.read_csv(campaigns_path)

In [172]:
# 데이터프레임의 크기 확인
print("데이터프레임의 크기:", campaigns.shape)

데이터프레임의 크기: (1907, 19)


In [173]:
# NaN 값을 적절한 기본값으로 대체
campaigns['started_at'].fillna('1970-01-01', inplace=True)
campaigns['finished_at'].fillna('1970-01-01', inplace=True)

In [174]:
# total_count, ab_test, warmup_mode, hour_limit, is_test, position에 대한 NaN 값 처리
campaigns['total_count'].fillna(0, inplace=True)  # 0으로 대체 또는 다른 적절한 값 사용
campaigns['ab_test'].fillna(False, inplace=True)  # False로 대체 또는 다른 적절한 값 사용
campaigns['warmup_mode'].fillna(False, inplace=True)  # False로 대체 또는 다른 적절한 값 사용
campaigns['hour_limit'].fillna(0, inplace=True)  # 0으로 대체 또는 다른 적절한 값 사용
campaigns['is_test'].fillna(False, inplace=True)  # False로 대체 또는 다른 적절한 값 사용
campaigns['position'].fillna(0, inplace=True)  # 0으로 대체 또는 다른 적절한 값 사용

# subject_with_personalization, subject_with_deadline, subject_with_emoji,
# subject_with_bonuses, subject_with_discount, subject_with_saleout를 boolean 값으로 변환
boolean_columns = ['subject_with_personalization', 'subject_with_deadline', 'subject_with_emoji',
                   'subject_with_bonuses', 'subject_with_discount', 'subject_with_saleout']
campaigns[boolean_columns] = campaigns[boolean_columns].astype(bool)

In [175]:
campaigns.tail()

,id,campaign_type,channel,topic,started_at,finished_at,total_count,ab_test,warmup_mode,hour_limit,subject_length,subject_with_personalization,subject_with_deadline,subject_with_emoji,subject_with_bonuses,subject_with_discount,subject_with_saleout,is_test,position
1902,179,transactional,email,profile updated,1970-01-01,1970-01-01,0.0,False,False,0.0,134.0,False,False,False,False,False,False,False,0.0
1903,35,transactional,email,order reminder,1970-01-01,1970-01-01,0.0,False,False,0.0,88.0,False,False,False,False,False,False,False,0.0
1904,57,transactional,email,order reminder,1970-01-01,1970-01-01,0.0,False,False,0.0,88.0,False,False,False,False,False,False,False,0.0
1905,56,transactional,email,order reminder,1970-01-01,1970-01-01,0.0,False,False,0.0,88.0,False,False,False,False,False,False,False,0.0
1906,237,transactional,email,wish list status,1970-01-01,1970-01-01,0.0,False,False,0.0,104.0,False,False,True,False,False,False,False,0.0


In [176]:
# 각 열의 데이터 유형 확인
column_types = campaigns.dtypes
# 결과 출력
print(column_types)

id                                int64
campaign_type                    object
channel                          object
topic                            object
started_at                       object
finished_at                      object
total_count                     float64
ab_test                            bool
warmup_mode                        bool
hour_limit                      float64
subject_length                  float64
subject_with_personalization       bool
subject_with_deadline              bool
subject_with_emoji                 bool
subject_with_bonuses               bool
subject_with_discount              bool
subject_with_saleout               bool
is_test                            bool
position                        float64
dtype: object


In [177]:
# # Elasticsearch 서버 연결
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [178]:
# Elasticsearch 서버에 ping 요청 보내기
if es.ping():
    print("Elasticsearch 서버에 성공적으로 연결되었습니다.")
else:
    print("Elasticsearch 서버에 연결하지 못했습니다.")

Elasticsearch 서버에 성공적으로 연결되었습니다.


In [179]:
# 인덱스가 이미 존재하는 경우 삭제
if es.indices.exists(index="e_campaigns"):
    es.indices.delete(index="e_campaigns")

In [180]:
# Elasticsearch 인덱스 매핑 설정
mapping = {
    "mappings": {
        "properties": {
            "id": {"type": "integer"},
            "campaign_type": {"type": "keyword"},
            "channel": {"type": "keyword"},
            "topic": {"type": "text"},
            "started_at": {"type": "date", "format": "yyyy-MM-dd'T'HH:mm:ss.SSSZZ"},
            "finished_at": {"type": "date", "format": "yyyy-MM-dd'T'HH:mm:ss.SSSZZ"},
            "total_count": {"type": "float"},
            "ab_test": {"type": "boolean"},
            "warmup_mode": {"type": "boolean"},
            "hour_limit": {"type": "float"},
            "subject_length": {"type": "float"},
            "subject_with_personalization": {"type": "boolean"},
            "subject_with_deadline": {"type": "boolean"},
            "subject_with_emoji": {"type": "boolean"},
            "subject_with_bonuses": {"type": "boolean"},
            "subject_with_discount": {"type": "boolean"},
            "subject_with_saleout": {"type": "boolean"},
            "is_test": {"type": "boolean"},
            "position": {"type": "float"}
        }
    }
}

In [181]:
es.indices.create(index="e_campaigns", body=mapping, ignore=400)

C:\Users\SEC\AppData\Local\Temp\ipykernel_26808\2770387805.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="e_campaigns", body=mapping, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'e_campaigns'})

In [188]:
# Elasticsearch에 이미 색인된 데이터 삭제
es.indices.delete(index="e_campaigns", ignore=[400, 404])

C:\Users\SEC\AppData\Local\Temp\ipykernel_26808\27307074.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="e_campaigns", ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [189]:
# 데이터프레임에서 Elasticsearch에 색인할 문서 생성
documents = []
for _, row in campaigns.iterrows():
    document = row.to_dict()

    # NaN이 아닌 경우에만 날짜를 ISO 형식으로 변환
    if not pd.isnull(document['started_at']):
        document['started_at'] = pd.to_datetime(document['started_at']).isoformat()
    if not pd.isnull(document['finished_at']):
        document['finished_at'] = pd.to_datetime(document['finished_at']).isoformat()

    # NaN을 None으로 대체
    for key, value in document.items():
        if pd.isna(value):
            document[key] = None

    documents.append({
        "_op_type": "index",
        "_index": "e_campaigns",
        "_source": document
    })

# 실패한 문서에 대한 정보 출력
for i, item in enumerate(documents):
    try:
        success, failed = bulk(es, [item])
    except BulkIndexError as e:
        print(f"Failed to index document {i + 1}: {e}")
        print(f"Error: {e.errors}")